Import necessary Libraries

In [1]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20181015225055-0000
Solving environment: done

## Package Plan ##

  environment location: /opt/ibm/conda/miniconda3

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    conda-4.5.11               |           py35_0         636 KB  conda-forge
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    ca-certificates-2018.8.24  |       ha4d7672_0         136 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The followin

Define Foursquare Credentials and Version

In [2]:
CLIENT_ID = 'ALV0DBYLFSFUILFVUA3UCEVLSWT43XPDBKEL5OLAUSZAGDUR' # your Foursquare ID
CLIENT_SECRET = 'P1L1YLKRZEZUNA0DMIOETZRBVZCUPHQVUAIGLCDXKNGORR3P' # your Foursquare Secret
VERSION = '20180822'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ALV0DBYLFSFUILFVUA3UCEVLSWT43XPDBKEL5OLAUSZAGDUR
CLIENT_SECRET:P1L1YLKRZEZUNA0DMIOETZRBVZCUPHQVUAIGLCDXKNGORR3P


Converting the address of university of alberta to its latitude and longitude coordinates.

In [25]:
address = 'University of Alberta,Edmonton, AB'
 # '102 North End Ave, New York, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


53.5238572 -113.522826652346


 define URL

In [20]:
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=ALV0DBYLFSFUILFVUA3UCEVLSWT43XPDBKEL5OLAUSZAGDUR&client_secret=P1L1YLKRZEZUNA0DMIOETZRBVZCUPHQVUAIGLCDXKNGORR3P&ll=53.60210575,-113.414564552021&v=20180822&radius=1000&limit=30'

In [29]:
import requests
results = requests.get(url).json()
'There are {} popular spots around University of Alberta.'.format(len(results['response']['groups'][0]['items']))

'There are 30  around University of Alberta.'

In [33]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-4b5ef940f964a52098a029e3-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d176941735',
    'name': 'Gym',
    'pluralName': 'Gyms',
    'primary': True,
    'shortName': 'Gym'}],
  'id': '4b5ef940f964a52098a029e3',
  'location': {'address': '13746 50 St.',
   'cc': 'CA',
   'city': 'Edmonton',
   'country': 'Canada',
   'crossStreet': 'Clareview',
   'distance': 398,
   'formattedAddress': ['13746 50 St. (Clareview)',
    'Edmonton AB T5A 5J6',
    'Canada'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 53.598540263307285,
     'lng': -113.41511493808355}],
   'lat': 53.598540263307285,
   'lng': -113.41511493808355,
   'postalCode': 'T5A 5J6',
   'state': 'AB'},
  'name': 'World Health',
  'photos': {'count': 

In [52]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,World Health,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",13746 50 St.,CA,Edmonton,Canada,Clareview,398,"[13746 50 St. (Clareview), Edmonton AB T5A 5J6...","[{'lat': 53.598540263307285, 'label': 'display...",53.598540,-113.415115,T5A 5J6,AB,4b5ef940f964a52098a029e3
1,Starbucks,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",14415 Miller Blvd. NW,CA,Edmonton,Canada,Manning Dr.,605,"[14415 Miller Blvd. NW (Manning Dr.), Edmonton...","[{'lat': 53.607535211574806, 'label': 'display...",53.607535,-113.414126,NaN,AB,4dc81d7a887717c880634ca0
2,Clareview Community Recreation Centre,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",3804 - 139 Ave NW,CA,Edmonton,Canada,NaN,817,"[3804 - 139 Ave NW, Edmonton AB T5Y 3E7, Canada]","[{'lat': 53.60224176231993, 'label': 'display'...",53.602242,-113.402185,T5Y 3E7,AB,549f59cd498e6de3ba6aa728
3,Costco Wholesale,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",13650 50 St NW,CA,Edmonton,Canada,NaN,709,"[13650 50 St NW, Edmonton AB T5A 4Y3, Canada]","[{'lat': 53.595827024769925, 'label': 'display...",53.595827,-113.416389,T5A 4Y3,AB,4b117059f964a520947c23e3
4,Cora's,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",4270 137 Ave,CA,Edmonton,Canada,NaN,642,"[4270 137 Ave, Edmonton AB, Canada]","[{'lat': 53.59783222191672, 'label': 'display'...",53.597832,-113.408026,NaN,AB,4f89a888e4b0e81f67a990ce
5,Starbucks,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",500 Manning Dr. NW,CA,Edmonton,Canada,in Safeway,522,"[500 Manning Dr. NW (in Safeway), Edmonton AB,...","[{'lat': 53.60104020828696, 'label': 'display'...",53.601040,-113.422275,NaN,AB,4ba41dfef964a520438338e3
6,Booster Juice,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",192 Manning Crossing Shopping Centre,CA,Edmonton,Canada,NaN,713,"[192 Manning Crossing Shopping Centre, Edmonto...","[{'lat': 53.60023072, 'label': 'display', 'lng...",53.600231,-113.424892,T5A 5A1,AB,4cddc10a70bba1cd45db5bc4
7,Red Robin Gourmet Burgers and Brews,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",4970 137th Avenue Northwest,CA,Edmonton,Canada,NaN,520,"[4970 137th Avenue Northwest, Edmonton AB T5Y ...","[{'lat': 53.59763347894459, 'label': 'display'...",53.597633,-113.412244,T5Y 2V4,AB,4b521eaef964a5207e6927e3
8,Bulk Barn,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",13846 40 St.,CA,Edmonton,Canada,NaN,654,"[13846 40 St., Edmonton AB T5Y 3E6, Canada]","[{'lat': 53.59971760411338, 'label': 'display'...",53.599718,-113.405508,T5Y 3E6,AB,4c38e72918e72d7fca4419f5
9,Tim Hortons,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",420 Manning Crossing Nw,CA,Edmonton,Canada,near Smitty's & Safeway on 50th St. & 137th Ave,447,[420 Manning Crossing Nw (near Smitty's & Safe...,"[{'lat': 53.6007216613977, 'label': 'display',...",53.600722,-113.420928,T5A 5A1,AB,4b154feef964a520c7aa23e3


In [58]:
new=dataframe_filtered[['name']]
new

,name
0,World Health
1,Starbucks
2,Clareview Community Recreation Centre
3,Costco Wholesale
4,Cora's
5,Starbucks
6,Booster Juice
7,Red Robin Gourmet Burgers and Brews
8,Bulk Barn
9,Tim Hortons


In [69]:
new1=dataframe_filtered.groupby('name').count()
new1['categories']

name
A&W Canada                               1
Best Buy Clareview                       1
Booster Juice                            1
Boston Pizza                             2
Bulk Barn                                1
Clareview Community Recreation Centre    1
Cora's                                   1
Costco Wholesale                         1
Dollarama                                1
Earls Restaurant & Bar                   1
Goodwill Industries of Alberta           1
Hong Huong Restaurant                    1
Kal Tire                                 1
Michaels                                 1
Oodle Noodle                             1
PetSmart                                 1
Pizza Hut                                1
Red Robin Gourmet Burgers and Brews      1
Rexall                                   1
Safeway                                  1
Shoppers Drug Mart                       1
Starbucks                                2
Subway                                   1
Swiss 